In [1]:
using Pkg
# cd("D:\\Home\\Git\\Oceananigans.jl")
cd("C:\\Users\\Ali\\Documents\\Git\\Oceananigans.jl\\")
Pkg.activate(".");

In [2]:
using Statistics

using Oceananigans, Oceananigans.Operators

In [3]:
∇²

UndefVarError: UndefVarError: ∇² not defined

In [4]:
∇²!

UndefVarError: UndefVarError: ∇²! not defined

In [5]:
@inline incmod1(a, n) = a == n ? one(a) : a + 1
@inline decmod1(a, n) = a == 1 ? n : a - 1

function ∇²(f)
    Nx, Ny, Nz = size(f)
    ∇²f = zeros(Nx, Ny, Nz)
    for k in 2:(Nz-1), j in 1:Ny, i in 1:Nx
       ∇²f[i, j, k] = f[incmod1(i, Nx), j, k] + f[decmod1(i, Nx), j, k] + f[i, incmod1(j, Ny), k] + f[i, decmod1(j, Ny), k] + f[i, j, k+1] + f[i, j, k-1] - 6*f[i, j, k]
    end
    for j in 1:Ny, i in 1:Nx
        ∇²f[i, j,   1] = -(f[i, j,     1] - f[i, j,   2]) + f[incmod1(i, Nx), j,   1] + f[decmod1(i, Nx), j,   1] + f[i, incmod1(j, Ny),   1] + f[i, decmod1(j, Ny),   1] - 4*f[i, j,   1]
        ∇²f[i, j, end] =  (f[i, j, end-1] - f[i, j, end]) + f[incmod1(i, Nx), j, end] + f[decmod1(i, Nx), j, end] + f[i, incmod1(j, Ny), end] + f[i, decmod1(j, Ny), end] - 4*f[i, j, end]
    end
    ∇²f
end

∇² (generic function with 1 method)

In [27]:
@inline incmod1(a, n) = a == n ? one(a) : a + 1
@inline decmod1(a, n) = a == 1 ? n : a - 1

function ∇²!(g::RegularCartesianGrid, f::CellField, ∇²f::CellField)
    for k in 2:(g.Nz-1), j in 1:g.Ny, i in 1:g.Nx
        ∇²f.data[i, j, k] = f.data[incmod1(i, g.Nx), j, k] + f.data[decmod1(i, g.Nx), j, k]
                          +  f.data[i, incmod1(j, g.Ny), k] + f.data[i, decmod1(j, g.Ny), k]
                          +  f.data[i, j, k+1]              + f.data[i, j, k-1]
                          -6*f.data[i, j, k]
    end
#     for j in 1:g.Ny, i in 1:g.Nx
#         ∇²f.data[i, j,   1] = (f.data[i, j, 2] - f.data[i, j, 1]) / g.Δz
#                             + (f.data[incmod1(i, g.Nx), j, 1] - 2*f.data[i, j, 1] + f.data[decmod1(i, g.Nx), j, 1]) / g.Δx
#                             + (f.data[i, incmod1(j, g.Ny), 1] - 2*f.data[i, j, 1] + f.data[i, decmod1(j, g.Ny), 1]) / g.Δy
#         ∇²f.data[i, j, end] = (f.data[i, j, end-1] - f.data[i, j, end]) / g.Δz
#                             + (f.data[incmod1(i, g.Nx), j, end] - 2*f.data[i, j, end] + f.data[decmod1(i, g.Nx), j, end]) / g.Δx
#                             + (f.data[i, incmod1(j, g.Ny), end] - 2*f.data[i, j, end] + f.data[i, decmod1(j, g.Ny), end]) / g.Δy
#     end
    nothing
end

∇²! (generic function with 1 method)

In [28]:
N = (20, 20, 20)
L = (20, 20, 20)
g = RegularCartesianGrid(N, L; dim=3, FloatType=Float64)

3-dimensional (Float64) regular Cartesian grid
(Nx, Ny, Nz) = (20, 20, 20)
(Lx, Ly, Lz) = (20.0, 20.0, 20.0)
(Δx, Δy, Δz) = (1.0, 1.0, 1.0)

In [29]:
f = CellField(g)
f.data .= rand(Float64, size(g));

In [30]:
∇²f1 = ∇²(f.data);

∇²f2 = CellField(g)
∇²!(g, f, ∇²f2)

∇²f3 = CellField(g)
∇²f3.data .= ∇²(f.data);

In [31]:
@show size(∇²f1)
∇²f1[:, :, 5]

size(∇²f1) = (20, 20, 20)


20×20 Array{Float64,2}:
  1.79812    2.54177    0.0610277  …  -2.74096     0.17731    1.18541 
 -2.32942    1.24371   -2.26241        1.5262      1.03041   -1.43511 
 -1.08675   -0.932096   0.447928      -0.924375    1.21192   -0.769011
  0.154511  -1.74215   -0.159156       2.46809    -1.70094   -1.79355 
 -3.14077    0.690266   1.86206        0.0404841  -1.50706    2.78682 
  3.40619   -1.31483   -2.64906    …  -2.31082     2.29272    1.45416 
  2.76233   -1.08212    2.47981        1.58365     0.860356  -3.69797 
 -1.33682    1.81293   -2.70391       -1.40254    -1.54075   -0.583683
 -0.584854   0.904871   0.0338601      1.54212    -2.50943    2.56166 
 -1.97883    0.510279   0.540942      -2.97782     1.90436    0.643408
  4.09431   -0.55056    0.429223   …   3.26721    -1.3613    -2.72816 
  1.20244   -2.63958    1.64226       -1.82849     3.35888   -1.39699 
 -1.01281    1.79883   -1.1547         2.64936    -3.89167    3.22053 
 -1.7524    -2.87885    1.49337        0.0838958  -1.

In [32]:
@show size(∇²f2.data)
∇²f2.data[:, :, 5]

size(∇²f2.data) = (20, 20, 20)


20×20 Array{Float64,2}:
  1.79812    2.54177    0.0610277  …  -2.74096     0.17731    1.18541 
 -2.32942    1.24371   -2.26241        1.5262      1.03041   -1.43511 
 -1.08675   -0.932096   0.447928      -0.924375    1.21192   -0.769011
  0.154511  -1.74215   -0.159156       2.46809    -1.70094   -1.79355 
 -3.14077    0.690266   1.86206        0.0404841  -1.50706    2.78682 
  3.40619   -1.31483   -2.64906    …  -2.31082     2.29272    1.45416 
  2.76233   -1.08212    2.47981        1.58365     0.860356  -3.69797 
 -1.33682    1.81293   -2.70391       -1.40254    -1.54075   -0.583683
 -0.584854   0.904871   0.0338601      1.54212    -2.50943    2.56166 
 -1.97883    0.510279   0.540942      -2.97782     1.90436    0.643408
  4.09431   -0.55056    0.429223   …   3.26721    -1.3613    -2.72816 
  1.20244   -2.63958    1.64226       -1.82849     3.35888   -1.39699 
 -1.01281    1.79883   -1.1547         2.64936    -3.89167    3.22053 
 -1.7524    -2.87885    1.49337        0.0838958  -1.

In [33]:
@show size(∇²f3.data)
∇²f3.data[:, :, 5]

size(∇²f3.data) = (20, 20, 20)


20×20 Array{Float64,2}:
  1.79812    2.54177    0.0610277  …  -2.74096     0.17731    1.18541 
 -2.32942    1.24371   -2.26241        1.5262      1.03041   -1.43511 
 -1.08675   -0.932096   0.447928      -0.924375    1.21192   -0.769011
  0.154511  -1.74215   -0.159156       2.46809    -1.70094   -1.79355 
 -3.14077    0.690266   1.86206        0.0404841  -1.50706    2.78682 
  3.40619   -1.31483   -2.64906    …  -2.31082     2.29272    1.45416 
  2.76233   -1.08212    2.47981        1.58365     0.860356  -3.69797 
 -1.33682    1.81293   -2.70391       -1.40254    -1.54075   -0.583683
 -0.584854   0.904871   0.0338601      1.54212    -2.50943    2.56166 
 -1.97883    0.510279   0.540942      -2.97782     1.90436    0.643408
  4.09431   -0.55056    0.429223   …   3.26721    -1.3613    -2.72816 
  1.20244   -2.63958    1.64226       -1.82849     3.35888   -1.39699 
 -1.01281    1.79883   -1.1547         2.64936    -3.89167    3.22053 
 -1.7524    -2.87885    1.49337        0.0838958  -1.

In [34]:
@show minimum(∇²f1), minimum(∇²f2.data);
@show maximum(∇²f1), maximum(∇²f2.data);
@show mean(∇²f1), mean(∇²f2.data);
@show sum(∇²f1), sum(∇²f2.data);

(minimum(∇²f1), minimum(∇²f2.data)) = (-5.1987200109170235, -5.1987200109170235)
(maximum(∇²f1), maximum(∇²f2.data)) = (4.721318862061617, 4.721318862061617)
(mean(∇²f1), mean(∇²f2.data)) = (3.7747582837255325e-18, 0.0001987835853370652)
(sum(∇²f1), sum(∇²f2.data)) = (3.019806626980426e-14, 1.5902686826965216)


In [35]:
x = 4 + 5

9

In [36]:
y = 4 + 5
  + 1 + 1

2

In [39]:
@show y;

y = 9


In [40]:
z = 4 + 5 +
    1 + 1

11

In [42]:
@show z;

z = 11
